<a href="https://colab.research.google.com/github/jinseriouspark/commerce-llm/blob/main/evaluation_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install huggingface-hub

In [2]:
# 현재 경로에 모델 저장
# 만약 huggingface-cli 를 활용해서 저장한다면, HF -> hubpython library -> download files -> download from cli 순으로 진행하는 것을 권장
!huggingface-cli download TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF capybarahermes-2.5-mistral-7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF/resolve/main/capybarahermes-2.5-mistral-7b.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpahqn6dh7
(…)apybarahermes-2.5-mistral-7b.Q4_K_M.gguf: 100% 4.37G/4.37G [00:12<00:00, 340MB/s]
./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf


In [3]:
!pip3 install hf_transfer
# 환경변수 HF_HUB_ENABLE_HF_TRANSFER = 1
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF capybarahermes-2.5-mistral-7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 26.0 MB/s eta 0:00:00
downloading https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF/resolve/main/capybarahermes-2.5-mistral-7b.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpepwwn6jw
(…)apybarahermes-2.5-mistral-7b.Q4_K_M.gguf: 100% 4.37G/4.37G [00:20<00:00, 213MB/s]
./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf


In [4]:
# Base ctransformers with no GPU acceleration
!pip install llama-cpp-python
# With NVidia CUDA acceleration
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 40.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.60-cp310-cp310-manylinux_2_35_x86_64.whl size=2983790 sha256=33320b5e13c9c1a978c1664bed59d7166b5838ec719ad1d790070b920bbe0b37
  Stored in directory: /root/.cache/pip/wheels/ad/7b/02/5200ea3612eca540182654a0b72f0b2a90c0f32d1633c931e4
Successfully built llama-cpp-python


In [5]:
from llama_cpp import Llama

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
  n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
)

# Simple inference example
output = llm(
  "<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)

# Chat Completion API

llm = Llama(model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf", chat_format="llama-2")  # Set chat_format according to the model you are using
llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "You are a story writing assistant."},
        {
            "role": "user",
            "content": "Write a story about llamas."
        }
    ]
)


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = argilla_capybarahermes-2.5-mistral-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:        

{'id': 'chatcmpl-2185a499-736b-4c85-b69c-4365b518cbc4',
 'object': 'chat.completion',
 'created': 1712585578,
 'model': './capybarahermes-2.5-mistral-7b.Q4_K_M.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '\n\nOnce upon a time, in the lush green fields of the Andes Mountains, there lived a herd of llamas. They were a happy and playful bunch, spending their days grazing on the fresh grass and frolicking with one another. The herd was led by an elderly llama named Llama-San, who had seen many seasons come and go.\n\nLlama-San was wise and kind, always looking out for his fellow llamas. He would often gather them around to share stories of the olden days when the fields were even greener and the grass sweeter. The young ones would listen intently, their eyes wide with wonder as Llama-San spun tales of adventure and friendship.\n\nOne day, a terrible drought struck the land. The once lush green fields turned brown and dry, and the grass became sparse 